In [27]:
!pip install -qU openai
!pip install -qU pinecone
!pip install -qU google-generativeai
!pip install -qU sentence-transformers

In [47]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["PINECONE_API_KEY"] = userdata.get("PINECONE_API_KEY")
os.environ["PINECONE_ENV"] = userdata.get("PINECONE_ENV")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")


In [48]:
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "digital-gold-index"
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region=os.environ["PINECONE_ENV"]
    )
)

index = pc.Index(index_name)


In [49]:
with open("/content/business_doc.txt", "r") as f:
    document = f.read()
chunks = [chunk.strip() for chunk in document.split("\n\n") if chunk.strip()]


In [50]:
for i, chunk in enumerate(chunks):
    embedding = embedding_model.encode(chunk).tolist()
    index.upsert([(f"doc-{i}", embedding, {"text": chunk})])


In [51]:
from typing import List

def search_pinecone(query: str, top_k: int = 3) -> List[str]:
    query_embedding = embedding_model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    return [match["metadata"]["text"] for match in results["matches"]]

# from sentence_transformers import SentenceTransformer
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# # Embed and upload chunks
# for i, chunk in enumerate(chunks):
#     embedding = embedding_model.encode(chunk).tolist()
#     index.upsert([(f"doc-{i}", embedding, {"text": chunk})])


In [52]:
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def generate_answer(query: str) -> str:
    context = "\n".join(search_pinecone(query))

    prompt = f"""
You are a professional virtual assistant designed specifically to help users with business-related queries only.

Respond strictly to questions related to business operations, services, products, policies, pricing, support, transactions, hours, or anything clearly tied to a business context.

If the user asks a question that is personal, casual, philosophical, unrelated to business, or off-topic, politely respond:
"Sorry, I can only assist with business-related questions."

Use only the following context to answer the question:
{context}

Now, based on the above instructions and the provided context, answer the user's question.

User's Question:
{query}

Answer:
"""

    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

#----------------------------------------------------------------------------------------------------------------------------
# from typing import List

# def search_pinecone(query: str, top_k: int = 3) -> List[str]:
#     query_embedding = embedding_model.encode(query).tolist()
#     results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
#     return [match["metadata"]["text"] for match in results["matches"]]

# def generate_answer(query: str) -> str:
#     context = "\n".join(search_pinecone(query))

#     messages = [
#         {"role": "system", "content": "You are a helpful assistant for a digital gold business."},
#         {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{query}"}
#     ]

#     response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=messages
#     )

#     return response.choices[0].message.content


In [57]:
query = input("Ask your business-related question: ")
answer = generate_answer(query)
print("\nAnswer:", answer)


Ask your business-related question: How do I sell my gold?

Answer: Sell any amount of your digital gold instantly via our platform. Proceeds are credited to your bank account within 24 hours.

